<a href="https://colab.research.google.com/github/JSJeong-me/KOSA-GPT_Python-2023/blob/main/PDF-gpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

PDF 파일기반 질의응답 챗봇 (랭체인, 그라디오, ChatGPT)

In [ ]:
# !pip install openai # openai 라이브러리를 설치합니다.
# !pip install langchain # 랭체인 라이브러리를 설치합니다.

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = "" # 환경변수에 OPENAI_API_KEY를 설정합니다.

In [ ]:
from langchain.chat_models import ChatOpenAI

In [ ]:
# !pip install pypdf
# !pip install chromadb
# !pip install tiktoken

# 새 섹션

In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings.cohere import CohereEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores.elastic_vector_search import ElasticVectorSearch
from langchain.vectorstores import Chroma

In [ ]:
loader = PyPDFLoader("foundation-model.pdf")
documents = loader.load()

In [ ]:
text_splitter = CharacterTextSplitter(chunk_size=2000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

In [ ]:
texts

In [ ]:
embeddings = OpenAIEmbeddings()
vector_store = Chroma.from_documents(texts, embeddings)
retriever = vector_store.as_retriever(search_kwargs={"k": 2})

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQAWithSourcesChain

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)  # Modify model_name if you have access to GPT-4

chain = RetrievalQAWithSourcesChain.from_chain_type(
    llm=llm,
    chain_type="map_reduce",
    retriever = retriever,
    return_source_documents=True)

In [ ]:
query = "What is the foundation model?"
result = chain(query)
print(result)

{'question': 'What is the foundation model?', 'answer': 'The foundation model is a type of AI model that is trained on broad data using self-supervision at scale and can be adapted to a wide range of downstream tasks. It is characterized by its central yet incomplete nature and has achieved impressive achievements in AI, demonstrating remarkable performance, interpretability, robustness, controllability, and generalization. The five essential properties of a foundation model are expressivity, scalability, multimodality, memory capacity, and compositionality.\n', 'sources': 'foundation-model.pdf', 'source_documents': [Document(page_content='On the Opportunities and Risks of\nFoundation Models\nRishi Bommasani* Drew A. Hudson Ehsan Adeli Russ Altman Simran Arora\nSydney von Arx Michael S. Bernstein Jeannette Bohg Antoine Bosselut Emma Brunskill\nErik Brynjolfsson Shyamal Buch Dallas Card Rodrigo Castellon Niladri Chatterji\nAnnie Chen Kathleen Creel Jared Quincy Davis Dorottya Demszky Ch

In [ ]:
result

In [ ]:
result['answer']

'The foundation model is a type of AI model that is trained on broad data using self-supervision at scale and can be adapted to a wide range of downstream tasks. It is characterized by its central yet incomplete nature and has achieved impressive achievements in AI, demonstrating remarkable performance, interpretability, robustness, controllability, and generalization. The five essential properties of a foundation model are expressivity, scalability, multimodality, memory capacity, and compositionality.\n'

In [ ]:
result['sources']

'foundation-model.pdf'

In [ ]:
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

system_template="""Use the following pieces of context to answer the users question shortly.
Given the following summaries of a long document and a question, create a final answer with references ("SOURCES"), use "SOURCES" in capital letters regardless of the number of sources.
If you don't know the answer, just say that "I don't know", don't try to make up an answer.
----------------
{summaries}

You MUST answer in Korean and in Markdown format:"""

messages = [
    SystemMessagePromptTemplate.from_template(system_template),
    HumanMessagePromptTemplate.from_template("{question}")
]

prompt = ChatPromptTemplate.from_messages(messages)

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQAWithSourcesChain

chain_type_kwargs = {"prompt": prompt}

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)  # Modify model_name if you have access to GPT-4

chain = RetrievalQAWithSourcesChain.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever = retriever,
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs
)

In [ ]:
query = "What is the proposal of foundation model?"
result = chain(query)
print(result)

{'question': 'What is the proposal of foundation model?', 'answer': '기초 모델의 제안은 다양한 하위 작업에 적용할 수 있는 광범위한 데이터로 훈련된 모델을 개발하는 것입니다. 이러한 모델은 BERT, DALL-E, GPT-3 등으로 알려진 모델들을 포함하며, 언어, 비전, 로봇 조작, 추론, 인간 상호작용 등 다양한 능력을 갖추고 있습니다. 이 보고서는 기초 모델의 기회와 위험에 대해 상세히 설명하고 있으며, 모델의 능력, 기술 원칙, 응용 분야, 사회적 영향 등을 다루고 있습니다. 또한, 기초 모델의 결함이 하위 작업에 적용된 모든 모델에 상속된다는 주의가 필요하다고 언급하고 있습니다. 이러한 질문들에 대한 명확한 이해와 연구를 위해 깊은 학문적 협력이 필요하다고 강조하고 있습니다. (', 'sources': 'foundation-model.pdf)', 'source_documents': [Document(page_content='On the Opportunities and Risks of\nFoundation Models\nRishi Bommasani* Drew A. Hudson Ehsan Adeli Russ Altman Simran Arora\nSydney von Arx Michael S. Bernstein Jeannette Bohg Antoine Bosselut Emma Brunskill\nErik Brynjolfsson Shyamal Buch Dallas Card Rodrigo Castellon Niladri Chatterji\nAnnie Chen Kathleen Creel Jared Quincy Davis Dorottya Demszky Chris Donahue\nMoussa Doumbouya Esin Durmus Stefano Ermon John Etchemendy Kawin Ethayarajh\nLi Fei-Fei Chelsea Finn Trevor Gale Lauren Gillespie Ka

In [ ]:
result['answer']

'기초 모델의 제안은 다양한 하위 작업에 적용할 수 있는 광범위한 데이터로 훈련된 모델을 개발하는 것입니다. 이러한 모델은 BERT, DALL-E, GPT-3 등으로 알려진 모델들을 포함하며, 언어, 비전, 로봇 조작, 추론, 인간 상호작용 등 다양한 능력을 갖추고 있습니다. 이 보고서는 기초 모델의 기회와 위험에 대해 상세히 설명하고 있으며, 모델의 능력, 기술 원칙, 응용 분야, 사회적 영향 등을 다루고 있습니다. 또한, 기초 모델의 결함이 하위 작업에 적용된 모든 모델에 상속된다는 주의가 필요하다고 언급하고 있습니다. 이러한 질문들에 대한 명확한 이해와 연구를 위해 깊은 학문적 협력이 필요하다고 강조하고 있습니다. ('

In [ ]:
result['source_documents']

In [ ]:
for doc in result['source_documents']:
    print('내용 : ' + doc.page_content[0:100].replace('\n', ' '))
    print('파일 : ' + doc.metadata['source'])
    print('페이지 : ' + str(doc.metadata['page']))

내용 : On the Opportunities and Risks of Foundation Models Rishi Bommasani* Drew A. Hudson Ehsan Adeli Russ
파일 : foundation-model.pdf
페이지 : 0
내용 : 12 Center for Research on Foundation Models (CRFM) Another complementary approach is to rely on volu
파일 : foundation-model.pdf
페이지 : 11


In [ ]:
!pip install gradio # 그라디오 라이브러리를 설치합니다.

In [ ]:
import gradio as gr

In [ ]:
def respond(message, chat_history):  # 채팅봇의 응답을 처리하는 함수를 정의합니다.

    result = chain(message)

    bot_message = result['answer']

    for i, doc in enumerate(result['source_documents']):
        bot_message += '[' + str(i+1) + '] ' + doc.metadata['source'] + '(' + str(doc.metadata['page']) + ') '

    chat_history.append((message, bot_message))  # 채팅 기록에 사용자의 메시지와 봇의 응답을 추가합니다.

    return "", chat_history  # 수정된 채팅 기록을 반환합니다.

with gr.Blocks() as demo:  # gr.Blocks()를 사용하여 인터페이스를 생성합니다.
    chatbot = gr.Chatbot(label="채팅창")  # '채팅창'이라는 레이블을 가진 채팅봇 컴포넌트를 생성합니다.
    msg = gr.Textbox(label="입력")  # '입력'이라는 레이블을 가진 텍스트박스를 생성합니다.
    clear = gr.Button("초기화")  # '초기화'라는 레이블을 가진 버튼을 생성합니다.

    msg.submit(respond, [msg, chatbot], [msg, chatbot])  # 텍스트박스에 메시지를 입력하고 제출하면 respond 함수가 호출되도록 합니다.
    clear.click(lambda: None, None, chatbot, queue=False)  # '초기화' 버튼을 클릭하면 채팅 기록을 초기화합니다.

demo.launch(debug=True)  # 인터페이스를 실행합니다. 실행하면 사용자는 '입력' 텍스트박스에 메시지를 작성하고 제출할 수 있으며, '초기화' 버튼을 통해 채팅 기록을 초기화 할 수 있습니다.

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>